## Mathusian-Verhulst-logistic-population-model


In [ ]:
## code from http://publish.illinois.edu/pillsburydoughcat/hw3-2/

import numpy as np #import library
 
n = 365 #number of times the code loops
dt = 1.0 #timestep
alpha=0.1#growth rate
K=2000.#carrying capacity
model = np.zeros((n,2)) #model, dependent
#and independent variables
 
model[0,0]=1#dependent initial condition
model[0,1]=100.#independent initial condition
 
for i in range(1,n):
     model[i,0]=model[i,0]+dt
     model[i,1]=model[i-1,1]\
     +dt*alpha*model[i-1,1]*(1-model[i-1,1]/K)

## Population simulation

In [ ]:
## http://publish.illinois.edu/pillsburydoughcat/hw3-2/hw3-supplementary-material/

%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
 
migration_rate = np.loadtxt('fish_migration.txt')
omegaMIN = 0.0009126
omegaONE = 0.000000001
rate_Coho = 5.6E6*(0.75)/365.25 #*0.75coho/fish * year/365.25 days
max_Coho = 5.6E6*(0.75)*3.0 #*0.75coho/fish * 3 years
arr_len = 60
dt = 1.0
model = np.zeros((arr_len,2))
rates = np.zeros((arr_len-1,3))
model[0,0]=1.
model[0,1]=max_Coho
 
for t in xrange(1,arr_len):
    model[t,0] = float(t)+1.
    rates[t-1,0] = rate_Coho
    rates[t-1,1] = -migration_rate[t-1,1]
    rates[t-1,2] = -(omegaMIN + omegaONE * model[t-1,1]) * model[t-1,1]
    model[t,1] = model[t-1,1] + dt * (sum(rates[t-1,:]) )
if model[t,1]<0:
    model[t,1]=0

plt.figure(1)
plt.plot(model[:,0],model[:,1])
plt.xlabel('time [days]')
plt.ylabel('populations [# of fish]')
plt.ylim(0,max(model[:,1]))
plt.figure(2)
plt.plot(model[:-1,0],rates[:,0],label='stocking')
plt.plot(model[:-1,0],-rates[:,1],label='migration out')
plt.plot(model[:-1,0],-rates[:,2],label='death rate')
plt.legend(loc='best')
plt.xlabel('time [days]')
plt.ylabel(r'$rate [\frac{fish}{days}]$')
plt.show()

In [ ]:
## Solution to Verhulst using 4th order Runga-Kutta

In [ ]:
&lt;br data-mce-bogus="1"&gt;
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
 
#numerical parameters
n = 31 #length of array
dt = 10. #time step
 
#Verhulst parameters
alpha = .1 #growth rate
K = 1000. #carrying capacity
 
#create arrays
N_numerical = np.zeros((n,3)) #make N arrray
N_analyical = np.zeros((n,1)) #make N arrray
t = np.linspace(0,(float(n)-1.)*dt,num=n) #make t array at constant interval
 
#initial condition
N_initial = 1.0
 
###FUNCTIONS
#Verhulst Analytical Solution
def verhulst_analytical(alpha,K,N_initial,t):
    N = (K * N_initial * np.exp(alpha*t))/(K+N_initial*(np.exp(alpha*t)-1.))
    return N
 
#Verhulst Numerical Integrator
def verhulst_rate(N,alpha,K):
    rate = alpha * N * (1.0 - N / K)
    return rate
 
#Euler
def euler(Nini,tini,alpha,K,dt,n):
    N = np.zeros((n))
    N[0] = Nini
    for i in xrange(1,n):
        rate = dt * verhulst_rate(N[i-1],alpha,K)
        N[i] = N[i-1] + rate
    return N
 
#2nd order runga kutta
def runga_kutta_2(Nini,tini,alpha,K,dt,n):
    N = np.zeros((n))
    N[0] = Nini
    for i in xrange(1,n):
        k1 = dt * verhulst_rate(N[i-1],alpha,K)
        k2 = dt * verhulst_rate((N[i-1]+k1/2.0),alpha,K)
        N[i] = N[i-1] + k2
    return N
 
#4th order runga kutta
def runga_kutta_4(Nini,tini,alpha,K,dt,n):
    N = np.zeros((n))
    N[0] = Nini
    for i in xrange(1,n):
        k1 = dt * verhulst_rate(N[i-1],alpha,K)
        k2 = dt * verhulst_rate((N[i-1]+k1/2.0),alpha,K)
        k3 = dt * verhulst_rate((N[i-1]+k2/2.0),alpha,K)
        k4 = dt * verhulst_rate((N[i-1]+k3),alpha,K)
        N[i] = N[i-1] + k1/6.0 + k2/3.0 + k3/3.0 + k4/6.0
    return N
 
#euler
N_numerical[:,0] = euler(N_initial,0,alpha,K,dt,n)
 
#2nd order runga kutta    
N_numerical[:,1] = runga_kutta_2(N_initial,0,alpha,K,dt,n)
 
#4th order runga kutta    
N_numerical[:,2] = runga_kutta_4(N_initial,0,alpha,K,dt,n)
     
#analytical soln
N_analytical = verhulst_analytical(alpha,K,N_initial,t)
 
plt.figure(1,facecolor = 'white')
plt.plot(t,N_numerical[:,0],label='Euler Method')
plt.plot(t,N_numerical[:,1],label=r'$2^{nd}$ Order Runga Kutta')
plt.plot(t,N_numerical[:,2],label=r'$4^{th}$ Order Runga Kutta')
plt.plot(t,N_analytical,label='Analytical Solution',ls='--',lw = 2)
plt.legend(loc='best')
plt.xlim(0,250)
plt.xlabel('time [T]')
plt.ylabel('populations [# of individuals]')
plt.savefig('Euler_and_2_4Runga.png')
plt.show()

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random
 
#Verhulst parameters
alpha = .1 #growth rate
K = 1000. #carrying capacity
 
#initial condition
N_initial = 1.0
runs = 100
num = int(K)-int(N_initial)+1
 
N = np.linspace(N_initial,K,num)
T = np.zeros((num,runs))
rnd = np.random.rand(num,runs)
 
T_avg = np.zeros((int(K)-int(N_initial)+1))
 
plt.figure(1,facecolor='white')
 
for i in xrange(0,runs):
    rate = alpha * N[1:] * (1.0 - N[1:] / K)
    dt = - np.log(1.0 - rnd[1:,i]) / rate
    T[1:,i] = np.cumsum(dt)
    T_avg += T[:,i]
    plt.plot(T,N,color ='b',alpha=0.1,lw = .5)
     
T_avg = T_avg / float(runs)
plt.plot(T_avg,N,color ='k',label ='Averaged Solution',lw =2)
N_analytical = (K * N_initial * np.exp(alpha*T_avg))/(K+N_initial*(np.exp(alpha*T_avg)-1.))    
plt.plot(T_avg,N_analytical,label='Analytical Solution',ls='--',lw = 2,color ='r')
plt.xlabel('t [days]')
plt.ylabel('N [population]')
plt.savefig('random.png')
plt.show()